<a href="https://colab.research.google.com/github/gabrielcaiaffa/Projeto-Futebol/blob/main/NLP_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing and Reading the data

In [71]:
#import libraries
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV 
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date, timedelta
import pandas as pd
import seaborn as sns
from google.colab import drive
! pip install pattern
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pattern
from pattern.en import lemma, lexeme

In [72]:
# pick data from google drive 
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/Noticias_Premier_League_all_teams/"
path2= "/content/gdrive/MyDrive/finaldataset.csv"

#path = '/content/gdrive/MyDrive/Noticias_Premier_League'
#path2 = '/content/gdrive/MyDrive/csv_files/finaldataset.csv'



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Preparing Data

## Create functions to use in DataFrame


In [169]:
# Create function to change de format of date
def change_data(v,x,d):
  if len(v[d][x])==8:
    year = '2'+'0'+v[d][x][6]+v[d][x][7]
    month = v[d][x][3]+ v[d][x][4]
    day = v[d][x][0]+ v[d][x][1]
    date = year+'-'+month+'-'+day
    return date
  elif len(v[d][x])==10:
      year = v[d][x][6]+v[d][x][7]+v[d][x][8]+v[d][x][9]
      month = v[d][x][3]+ v[d][x][4]
      day = v[d][x][0]+ v[d][x][1]
      date = year+'-'+month+'-'+day
      return date


In [170]:
# Create function to apply dummies
def dummies(resultado_d,x):
  if x == 'H':
    resultado_d.append(1)
  elif x=='D':
    resultado_d.append(2)
  elif x=='A':
    resultado_d.append(3)
    

In [171]:
# Create a function to pick news ten days ago the game 
def get_news(dataframe,x,timev,casa):
  time = dataframe.loc[x,casa]                    
  vetor_resultado = []
  if time in timev:
    vetor_noticias=[]
    arquivo_noticias = times[time]
    data_atual = date.fromisoformat(dataframe['Date'][x])
    for y in range (1,10):
      data_analisada = data_atual - timedelta(days=y)
      data = data_analisada.isoformat()
      n_atual =arquivo_noticias.loc[arquivo_noticias['date']==data]
      vetor = np.array(n_atual)
      for k in range (0,len(vetor)):
          texto = ''
          for h in range(3,6):
            texto = texto+''+str(vetor[k][h])
          vetor_noticias.append(texto)
    data_frame_vetor = pd.DataFrame(vetor_noticias)
    data_frame_vetor.drop_duplicates(inplace = True)
    vetor_resultado = np.array(data_frame_vetor)
  return vetor_resultado

##Create dataframes using created fuctions in data


In [172]:
# Create dataframe with date of games and replace abbreviations of teams
dfp = pd.read_csv(path2, index_col = [0])
dfp.replace('Man United','Manchester United',inplace=True)
dfp.replace ('Man City','Manchester City',inplace=True)
dfp.replace ('West Ham','West Ham United',inplace=True)
dfp.replace ('Chelsea','Chelsea FC',inplace=True)
dfp.replace ('Liverpool','Liverpool FC',inplace=True)
dfp.replace ('Fulham','Fulham FC',inplace=True)
dfp.replace ('Burnley','Burnley FC',inplace=True)
dfp.replace ('Southampton','Southampton FC',inplace=True)
dfp.replace ('Everton','Everton FC',inplace=True)
dfp.replace ('Arsenal','Arsenal FC',inplace=True)
dfp.replace ('Newcastle','Newcastle United',inplace=True)
dfp.replace ('West Brom','West Bromwich Albion',inplace=True)
dfp.replace ('Wolves','Wolverhampton Wanderers',inplace=True)
dfp.replace ('Tottenham','Tottenham Hotspur',inplace=True)
dfp.replace ('Norwich','Norwich City',inplace=True)
dfp.replace ('QPR','Queens Park Rangers',inplace=True)
dfp.replace ('Reading','Reading FC',inplace=True)
dfp.replace ('Stoke','Stoke City',inplace=True)
dfp.replace ('Sunderland','Sunderland AFC',inplace=True)
dfp.replace ('Leicester','Leicester City',inplace=True)
dfp.replace ('Bournemouth','AFC Bournemouth',inplace=True)
dfp.replace ('Middlesbrough','Middlesbrough FC',inplace=True)
dfp.replace ('Brighton','Brighton & Hove Albion',inplace=True)
dfp.replace ('Huddersfield','Huddersfield Town',inplace=True)

dfp


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,19/08/00,Charlton,Manchester City,4,0,H,17,8,14,4,6,6,13,12,1,2,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,0.0,0.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,0.0
1,19/08/00,Chelsea FC,West Ham United,4,2,H,17,12,10,5,7,7,19,14,1,2,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,5.0,9.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-4.0
2,19/08/00,Coventry,Middlesbrough FC,1,3,A,6,16,3,9,8,4,15,21,5,3,1,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0
3,19/08/00,Derby,Southampton FC,2,2,D,6,13,4,6,5,8,11,13,1,1,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,16.0,15.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,1.0
4,19/08/00,Leeds,Everton FC,2,0,H,17,12,8,6,6,4,21,20,1,3,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,3.0,13.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,26/07/2020,Leicester City,Manchester United,0,2,A,14,7,3,3,3,3,12,11,1,4,1,0,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0
7596,26/07/2020,Manchester City,Norwich City,5,0,H,31,5,10,4,9,0,7,4,1,1,0,0,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0
7597,26/07/2020,Newcastle United,Liverpool FC,1,3,A,3,14,2,6,2,4,11,5,1,0,0,0,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0
7598,26/07/2020,Southampton FC,Sheffield United,3,1,H,13,5,4,3,9,1,9,16,0,1,0,0,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0


In [173]:
# apply change data in dfp
for x in range (0,len(dfp)):
  dfp.loc[x,'Date']=change_data(dfp,x,'Date')

In [174]:
# Separate target
resultado = dfp['FTR']

In [175]:
# apply dummies in results
resultado_d = []
for y in resultado:
  dummies(resultado_d,y)

In [176]:
# Create Features Noticias Home and Noticias Away
noticias_home = []
noticias_away = []
for x in range(0,len(resultado_d)):
  noticias_home.append([])
  noticias_away.append([])

In [177]:
# Aplly features in DataFrame
dfp['Noticias Home']=noticias_home
dfp['Noticias Away']=noticias_away
dfp['FTR'] = resultado_d

In [178]:
# Create dataframes news of teams
team0=pd.read_csv(path + 'Arsenal FC.csv' )
team1=pd.read_csv(path + 'Aston Villa.csv' )
team2=pd.read_csv(path + 'Brighton & Hove Albion.csv' )
team3=pd.read_csv(path + 'Burnley FC.csv' )
team4=pd.read_csv(path + 'Chelsea FC.csv' )
team5=pd.read_csv(path + 'Crystal Palace.csv' )
team6=pd.read_csv(path + 'Everton FC.csv' )
team7=pd.read_csv(path + 'Fulham FC.csv' )
team8=pd.read_csv(path + 'Leeds United.csv' )
team9=pd.read_csv(path + 'Leicester City.csv' )
team10=pd.read_csv(path + 'Liverpool FC.csv' )
team11=pd.read_csv(path + 'Manchester City.csv' )
team12=pd.read_csv(path + 'Manchester United.csv' )
team13=pd.read_csv(path + 'Newcastle United.csv' )
team14=pd.read_csv(path + 'Sheffield United.csv' )
team15=pd.read_csv(path + 'Southampton FC.csv' )
team16=pd.read_csv(path + 'Tottenham Hotspur.csv' )
team17=pd.read_csv(path + 'West Bromwich Albion.csv' )
team18=pd.read_csv(path + 'West Ham United.csv' )
team19=pd.read_csv(path + 'Wolverhampton Wanderers.csv')
team20=pd.read_csv(path +'Cardiff City.csv')
team21=pd.read_csv(path +'Hull City.csv')
team22=pd.read_csv(path +'Swansea City.csv')
team23=pd.read_csv(path +'Wigan Athletic.csv')
team24=pd.read_csv(path +'Norwich City.csv')
team25=pd.read_csv(path +'Queens Park Rangers.csv')
team26=pd.read_csv(path +'Stoke City.csv')
team27=pd.read_csv(path +'Sunderland AFC.csv')
team28=pd.read_csv(path +'AFC Bournemouth.csv')
team29=pd.read_csv(path +'Watford FC.csv')
team30=pd.read_csv(path +'Middlesbrough FC.csv')
team31=pd.read_csv(path +'Huddersfield Town.csv')



In [179]:
# Create diconary relating teams with yours dataframes 
times = {'Arsenal FC':team0,'Aston Villa':team1,'Brighton & Hove Albion':team2,'Burnley FC':team3,'Chelsea FC':team4, 'Crystal Palace':team5,'Everton FC':team6,'Fulham FC':team7,'Leeds United':team8,'Leicester City':team9,'Liverpool FC':team10,'Manchester City':team11,'Manchester United':team12,'Newcastle United':team13,'Sheffield United':team14,'Southampton FC':team15,'Tottenham Hotspur':team16,'West Bromwich Albion':team17,'West Ham United':team18,'Wolverhampton Wanderers':team19
         ,'Cardiff City':team20,'Hull City':team21,'Swansea City':team22,'Wigan Athletic':team23,'Norwich City':team24,'Queens Park Rangers':team25,'Stoke City':team26,
         'Sunderland AFC':team27,'AFC Bournemouth':team28,'Watford FC':team29,'Middlesbrough FC':team30,'Huddersfield Town':team31}
times_vetor = list(times.keys())

In [180]:
# Create a Dataframe with games as from 2013
df = dfp.loc[4758:]
df.reset_index(drop = True, inplace= True)
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away
0,2013-01-01,Manchester City,Stoke City,3,0,1,17,4,9,3,7,2,6,9,2,2,0,0,34,18,16,14,1.950000,1.400000,L,W,W,D,W,D,L,D,D,W,1.0,14.0,20.0,LWWLD,WDDDW,7,9,0.900000,0.200000,0.550000,-0.100000,-13.0,[],[]
1,2013-01-01,Southampton FC,Arsenal FC,1,1,2,9,6,6,4,3,3,9,6,2,0,0,0,26,39,37,21,0.850000,1.650000,D,W,D,W,L,W,W,W,L,L,0.0,3.0,20.0,DDLWL,WWWWL,5,12,-0.550000,0.900000,-0.800000,-0.350000,-3.0,[],[]
2,2013-01-01,Swansea,Aston Villa,2,2,2,23,8,13,6,3,2,7,15,0,4,0,0,29,15,24,39,1.333333,0.857143,W,L,D,L,D,L,L,W,L,D,11.0,16.0,21.0,WDDLL,LLLWD,5,4,0.238095,-1.142857,0.476190,0.047619,-5.0,[],[]
3,2013-01-01,Tottenham Hotspur,Reading FC,3,1,1,30,7,20,4,12,3,9,9,0,0,0,0,36,22,26,37,1.714286,0.619048,W,W,W,D,D,L,W,L,L,L,4.0,0.0,21.0,WWDWL,WDLLL,10,4,0.476190,-0.714286,1.095238,0.285714,4.0,[],[]
4,2013-01-01,West Bromwich Albion,Fulham FC,1,2,3,17,10,10,5,10,5,14,6,0,0,0,0,28,30,25,36,1.571429,1.000000,L,L,W,D,W,L,D,L,L,W,10.0,9.0,21.0,LWWDL,LDLLW,7,4,0.142857,-0.285714,0.571429,0.142857,1.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,2020-07-26,Leicester City,Manchester United,0,2,3,14,7,3,3,3,3,12,11,1,4,1,0,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0,[],[]
2838,2020-07-26,Manchester City,Norwich City,5,0,1,31,5,10,4,9,0,7,4,1,1,0,0,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0,[],[]
2839,2020-07-26,Newcastle United,Liverpool FC,1,3,3,3,14,2,6,2,4,11,5,1,0,0,0,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0,[],[]
2840,2020-07-26,Southampton FC,Sheffield United,3,1,1,13,5,4,3,9,1,9,16,0,1,0,0,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0,[],[]


In [181]:
# apply data change in news
for x in times_vetor:
  time = times[x]
  for y in range(0,len(time)):
    time.loc[y,'date']=change_data(time,y,'date')

In [182]:
#Add HomeTeam and AwayTeam news
for x in range (0,len(df)):
    df['Noticias Home'][x]=get_news(df,x,times_vetor,'HomeTeam')
    df['Noticias Away'][x]=get_news(df,x,times_vetor,'AwayTeam') 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

# Finishing changes in dataframe and transformng news in lists with only one text 


In [186]:
# Transform the vectos of HomeTeam in a list with one text
for x in range(0,len(df)):
  textos = ' '
  for y in range(0,len(df.loc[x,'Noticias Home'])):
    textos = textos +''+df.loc[x,'Noticias Home'][y][0]
  df.loc[x,'Noticias Home'] = textos

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [188]:
# Transform the vectos of HomeTeam in a list with one text
for x in range(0,len(df)):
  textos = ' '
  for y in range(0,len(df.loc[x,'Noticias Away'])):
    textos = textos +''+df.loc[x,'Noticias Away'][y][0]
  df.loc[x,'Noticias Away'] = textos

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [198]:
# Function to apply Dummies in "HM"and "AM" features
def m_dummies(x,y,w):
    if x.loc[w,y] == 'W':
       x.loc[w,y]=1
    elif x.loc[w,y]=='D':
        x.loc[w,y]=2
    elif x.loc[w,y]=='L':
      x.loc[w,y]=3
    elif  x.loc[w,y]=='M':
       x.loc[w,y]=4

In [199]:
#Apply Dummies in "HM"and "AM" features
new_dummies= ['HM1', 'AM1', 'HM2','AM2', 'HM3', 'AM3', 'HM4', 'AM4', 'HM5', 'AM5']
for x in new_dummies:
  for y in range(0,len(df)):
     m_dummies(df,x,y)

In [200]:
df.reset_index(drop = True, inplace= True)

In [203]:
# Drop games whitout news
for x in range(0,len(df)):
  if df.loc[x,'Noticias Home']==' ' and df.loc[x,'Noticias Away']==' ':
      df.drop(index=x,inplace= True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Dividing data in train and test and apply tfidf in news

In [ ]:
# Features to use in training
columns = ['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'AM1', 'HM2',
       'AM2', 'HM3', 'AM3', 'HM4', 'AM4', 'HM5', 'AM5', 'HomeTeamLP', 'AwayTeamLP','HTGD', 'ATGD', 'DiffPts',
       'DiffFormPts', 'DiffLP','Noticias Home','Noticias Away']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[columns],df['FTR'], test_size=0.3, random_state=0,stratify=df['FTR'])

In [ ]:
X_train.reset_index(drop = True,inplace = True)
y_train.reset_index(drop = True,inplace = True)
X_test.reset_index(drop = True,inplace = True)
y_test.reset_index(drop = True,inplace = True)

# Create a fuction to generate a vector that will be use in TfidfVectorizer and apply it 

In [ ]:
def vetor_lemma(df):
# Create a vetor with news to use in TfidfVectorizer  
  vetor_news = []
  for x in range(0,len(df)):
     vetor_news.append(df['Noticias Home'][x])
     vetor_news.append(df['Noticias Away'][x])
     vector_news_clean = []
  # Clean text of vetor_news
  clean= ['0','1','2','3','4','5','6','7','8','9','@','©','&','/','\"','-']
  for x in range (0,len(vetor_news)):
      add = ''
      if vetor_news[x] is not np.nan:
          for y in range (0,len(vetor_news[x])):
              if(vetor_news[x][y] not in clean):
                  add = add + vetor_news[x][y]
      vector_news_clean.append(add)
  # Drop duplicates in vector_news_clean
  vetor = list(dict.fromkeys(vector_news_clean))
  vector_train =[]
  # Apply lemmatization
  for x in range (0,len(vetor)):
      vector_train.append(" ".join([lemma(wd) for wd in vetor[x].split()]))
  return vector_train

In [ ]:
news_train = vetor_lemma(X_train)

# Applying vectorizer

#### Vectorizer in train news

In [ ]:
# Fit vectorizer for train
vectorizer_train = TfidfVectorizer(stop_words={'english'},max_features=2000)
vectorizer_train.fit(news_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words={'english'}, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
train_home = vectorizer_train.transform(X_train['Noticias Home'])
train_away = vectorizer_train.transform(X_train['Noticias Away'])

In [ ]:
# Transforms vectors to dense form
train_home_d = train_home.todense()
train_away_d = train_away.todense()
X_train_ok = np.hstack((train_home_d,train_away_d))
y_train_ok = np.array(y_train)

#### Vectorizer in test news

In [ ]:
test_home = vectorizer_train.transform(X_test['Noticias Home'])
test_away = vectorizer_train.transform(X_test['Noticias Away'])

In [ ]:
# Transforms vectors to dense form
test_home_d = test_home.todense()
test_away_d = test_away.todense()
X_test_ok = np.hstack((test_home_d,test_away_d))
y_test_ok = np.array(y_train)

### Add news in format of vectores in training 

In [ ]:
# Create dataframe with matrix of news
dftrain = pd.DataFrame(X_train_ok)

In [ ]:
# Drop news in format of words
X_train.drop(columns=['Noticias Home','Noticias Away'],inplace=True)

In [ ]:
X_train =pd.concat((X_train,dftrain),ignore_index=True,axis=1)

In [ ]:
columns_historic = ['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'AM1', 'HM2',
       'AM2', 'HM3', 'AM3', 'HM4', 'AM4', 'HM5', 'AM5', 'HomeTeamLP', 'AwayTeamLP','HTGD', 'ATGD', 'DiffPts',
       'DiffFormPts', 'DiffLP']

In [ ]:
# back name for columns of historic
y = 0
for x in columns_historic:
    X_train.rename(columns={y:x},inplace=True)
    y=y+1

### Add news in format of vectores is test

In [ ]:
# Create dataframe with matrix of news
dftest = pd.DataFrame(X_test_ok)

In [ ]:
# Drop news in format of words
X_test.drop(columns=['Noticias Home','Noticias Away'],inplace=True)

In [ ]:
X_test =pd.concat((X_test,dftest),ignore_index=True,axis=1)

In [ ]:
# back name for columns of historic
y = 0
for x in columns_news:
    X_test.rename(columns={y:x},inplace=True)
    y=y+1

In [ ]:
X_test

,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,35,36,37,38,39,...,3995,3996,3997,3998,3999,4000,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,4011,4012,4013,4014,4015,4016,4017,4018,4019,4020,4021,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034
0,17,13,6,2,4,3,6,15,0,3,0,0,16,9,8,17,1.545455,0.363636,1,3,2,3,2,3,1,3,1,3,5.0,17.0,0.727273,-0.727273,1.181818,1.000000,-12.0,0.00000,0.000000,0.000000,0.000000,0.028002,...,0.007955,0.0,0.012270,0.086322,0.000000,0.006153,0.0,0.0,0.0,0.0,0.012142,0.007351,0.000000,0.000000,0.0,0.000000,0.015917,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.033359,0.0,0.016480,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.026279,0.0
1,29,8,7,2,13,3,11,7,3,2,0,1,42,26,42,52,1.257143,0.742857,2,3,3,3,2,3,3,2,1,3,13.0,0.0,0.000000,-0.742857,0.514286,0.114286,13.0,0.03593,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,15,8,3,3,10,7,12,13,2,4,0,0,43,20,16,30,2.047619,0.904762,2,3,2,2,1,3,1,3,3,2,6.0,8.0,1.285714,-0.476190,1.142857,0.285714,-2.0,0.00000,0.000000,0.005799,0.027856,0.000000,...,0.000000,0.0,0.009753,0.096061,0.006388,0.009781,0.0,0.0,0.0,0.0,0.028954,0.040904,0.008492,0.000000,0.0,0.000000,0.004217,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.097227,0.0,0.026199,0.0,0.000000,0.000000,0.0,0.042840,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,14,7,4,2,4,6,14,18,2,1,0,1,10,25,27,27,0.647059,1.294118,2,3,2,2,2,3,1,1,2,1,14.0,17.0,-1.000000,-0.117647,-0.647059,0.000000,-3.0,0.00000,0.000000,0.044340,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,15,10,3,4,7,9,7,17,3,4,0,0,20,18,17,21,1.375000,1.312500,3,2,1,3,2,1,3,1,1,3,12.0,6.0,0.187500,-0.187500,0.062500,0.000000,6.0,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.012919,0.0,0.000000,0.112146,0.000000,0.009992,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.019289,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.014029,0.018676,0.0,0.009029,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.017505,0.000000,0.000000,0.00000,0.0,0.0,0.023348,0.000000,0.014225,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,26,7,9,4,5,5,6,9,1,1,0,0,24,15,17,28,1.666667,0.777778,1,1,1,3,1,3,2,1,2,3,5.0,17.0,0.388889,-0.722222,0.888889,0.277778,-12.0,0.00000,0.008709,0.000000,0.008502,0.005914,...,0.000000,0.0,0.000000,0.071665,0.012132,0.009287,0.0,0.0,0.0,0.0,0.027492,0.000000,0.000000,0.000000,0.0,0.000000,0.064072,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.008393,0.0,0.012438,0.0,0.018488,0.000000,0.0,0.008135,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.024420,0.000000,0.0
685,13,8,6,3,3,3,8,13,3,4,0,0,19,15,38,34,0.600000,0.400000,3,3,3,2,3,2,3,3,3,2,16.0,17.0,-0.950000,-0.950000,0.200000,-0.150000,-1.0,0.00000,0.000000,0.000000,0.014621,0.000000,...,0.014594,0.0,0.000000,0.063342,0.000000,0.022574,0.0,0.0,0.0,0.0,0.011137,0.000000,0.000000,0.000000,0.0,0.000000,0.029200,0.0,0.000000,0.0,0.015848,0.000000,0.0,0.020399,0.0,0.000000,0.0,0.044937,0.000000,0.0,0.049435,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.016069,0.0
686,16,9,10,5,5,10,6,8,0,0,0,0,41,21,30,33,1.62

# Create fuction to apply Logistic Regression with cross_validation 

In [ ]:
def reg_log_cross(X_train,y_train,X_test,y_test):
  log_reg = LogisticRegression(random_state = 42, max_iter = 1000, multi_class = 'ovr')
  # defining the search space
  space = dict()
  space['solver'] = ['lbfgs', 'newton-cg', 'liblinear']
  space['penalty'] = ['l1', 'l2', 'elasticnet']
  space['C'] = np.arange(10**-4, 20, 0.01)
  # training the model and fitting our data to it
  rscv = RandomizedSearchCV(log_reg, space, cv = 3, n_jobs = -1, n_iter = 100,verbose=2)
  result = rscv.fit(X_train,y_train)
  #best C and regularization
  C = rscv.best_params_
  c = C['C']
  l_penalty = C['penalty']
  l_solver = C['solver']
  #printing c (best hyperparameter) and penalty
  print("C: ", c, "\nPenalty: ", l_penalty)
  print("Solver: ", l_solver)
  print('--------------------------')
  reg_log= LogisticRegression(solver= l_solver,random_state=0,penalty= l_penalty,C= c)
  reg_log.fit(X_train,y_train)
  predictions_train = reg_log.predict(X_train)
  predictions_test = reg_log.predict(X_test)
  print(confusion_matrix(y_test,predictions_test))  
  print(classification_report(y_test,predictions_test))  
  print('Acurracy on train: {}'.format(accuracy_score(y_train, predictions_train)))
  print('Acurracy on test: {}'.format(accuracy_score(y_test, predictions_test)))

  

# Apply Logistic Regression only with news

In [ ]:
X_train_news = X_train.drop(columns_historic, axis=1)

In [ ]:
X_test_news = X_test.drop(columns_historic, axis=1)

In [ ]:
reg_log_cross(X_train_news,y_train,X_test_news,y_test)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.5min finished


C:  1.3001 
Penalty:  l2
Solver:  newton-cg
--------------------------
[[255   2  61]
 [109   7  40]
 [112  13  90]]
              precision    recall  f1-score   support

           1       0.54      0.80      0.64       318
           2       0.32      0.04      0.08       156
           3       0.47      0.42      0.44       215

    accuracy                           0.51       689
   macro avg       0.44      0.42      0.39       689
weighted avg       0.47      0.51      0.45       689

Acurracy on train: 0.6631382316313823
Acurracy on test: 0.5108853410740203


# Apply Logistic Regression only historic

In [ ]:
X_train_wn = X_train[columns_historic]

In [ ]:
X_test_wn = X_test[columns_historic]

In [ ]:
reg_log_cross(X_train_wn,y_train,X_test_wn,y_test)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 206 tasks      | elapsed:   41.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.0min finished


C:  0.6201 
Penalty:  l2
Solver:  newton-cg
--------------------------
[[265  22  31]
 [ 91  17  48]
 [ 45  26 144]]
              precision    recall  f1-score   support

           1       0.66      0.83      0.74       318
           2       0.26      0.11      0.15       156
           3       0.65      0.67      0.66       215

    accuracy                           0.62       689
   macro avg       0.52      0.54      0.52       689
weighted avg       0.57      0.62      0.58       689

Acurracy on train: 0.6475716064757161
Acurracy on test: 0.6182873730043541


#Apply Logistic Regression with news and historic 

In [ ]:
reg_log_cross(X_train,y_train,X_test,y_test)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 25.8min finished


C:  0.0301 
Penalty:  l1
Solver:  liblinear
--------------------------
[[278   3  37]
 [ 97   7  52]
 [ 52   4 159]]
              precision    recall  f1-score   support

           1       0.65      0.87      0.75       318
           2       0.50      0.04      0.08       156
           3       0.64      0.74      0.69       215

    accuracy                           0.64       689
   macro avg       0.60      0.55      0.51       689
weighted avg       0.61      0.64      0.58       689

Acurracy on train: 0.6382316313823163
Acurracy on test: 0.6444121915820029


#Results of acurracy on test

###Only news: 0.5108853410740203

###Only historic: 0.6182873730043541

###News and historic:  0.6444121915820029